In [1]:
def replace_name(text):
    # 替换 [name] 和 [NAME] 为 Jane
    text = text.replace('[name]', 'Jane')
    text = text.replace('[NAME]', 'Jane')
    text = text.replace('[age]', '50')
    text = text.replace('[AGE]', '50')
    return text

In [2]:
import json
from refchecker import LLMExtractor

data_path1 = 'data/raw.json'
data_path2 = 'data/claims.json'

def replace_name(text):
    # 替换 [name] 和 [NAME] 为 Jane
    text = text.replace('[name]', 'Jane')
    text = text.replace('[NAME]', 'Jane')
    text = text.replace('[age]', '50')
    text = text.replace('[AGE]', '50')
    return text

def extract(data_path1, data_path2):
    extractor = LLMExtractor(
    claim_format='triplet', 
    model='bedrock/meta.llama3-1-70b-instruct-v1:0',
    batch_size=50,
    )
    data = json.load(open(data_path1))

    batch_responses_ref = []
    batch_responses_t1= []
    batch_responses_t2 = [] 

    for d in data:
        ref = d["reference"]
        text1 = d["text1"]
        text2 = d["text2"]

        batch_responses_ref.append(replace_name(ref))
        batch_responses_t1.append(replace_name(text1))
        batch_responses_t2.append(replace_name(text2)) 

    response_extract_results = extractor.extract(
        batch_responses=batch_responses_ref
    )
    response_claims_ref = [[c.content for c in res.claims] for res in response_extract_results]
    for d_id,d in enumerate(data):
        if len(data) == 1:
            d['claims_ref'] = response_claims_ref
        else:
            d['claims_ref'] = response_claims_ref[int(d_id)]

    response_extract_results = extractor.extract(
        batch_responses=batch_responses_t1
    )
    response_claims_t1 = [[c.content for c in res.claims] for res in response_extract_results]
    for d_id,d in enumerate(data):
        if len(data) == 1:
            d['claims_t1'] = response_claims_t1
        else:
            d['claims_t1'] = response_claims_t1[int(d_id)]

    response_extract_results = extractor.extract(
        batch_responses=batch_responses_t2
    )
    response_claims_t2 = [[c.content for c in res.claims] for res in response_extract_results]
    for d_id,d in enumerate(data):
        if len(data) == 1:
            d['claims_t2'] = response_claims_t2
        else:
            d['claims_t2'] = response_claims_t2[int(d_id)]
    
    with open(data_path2, 'w') as f:  
        json.dump(data, f, indent=4)

extract(data_path1, data_path2)      

/opt/conda/envs/refchecker/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


  0%|          | 0/8 [00:00<?, ?it/s]

litellm.APIConnectionError: [Errno 9] Bad file descriptor
Traceback (most recent call last):
  File "/opt/conda/envs/refchecker/lib/python3.10/site-packages/httpx/_transports/default.py", line 69, in map_httpcore_exceptions
    yield
  File "/opt/conda/envs/refchecker/lib/python3.10/site-packages/httpx/_transports/default.py", line 233, in handle_request
    resp = self._pool.handle_request(req)
  File "/opt/conda/envs/refchecker/lib/python3.10/site-packages/httpcore/_sync/connection_pool.py", line 216, in handle_request
    raise exc from None
  File "/opt/conda/envs/refchecker/lib/python3.10/site-packages/httpcore/_sync/connection_pool.py", line 196, in handle_request
    response = connection.handle_request(
  File "/opt/conda/envs/refchecker/lib/python3.10/site-packages/httpcore/_sync/connection.py", line 101, in handle_request
    return self._connection.handle_request(request)
  File "/opt/conda/envs/refchecker/lib/python3.10/site-packages/httpcore/_sync/http11.py", line 143, in 

100%|██████████| 8/8 [04:20<00:00, 32.51s/it]


In [3]:
from refchecker import LLMChecker
import json
d1 = 'data/claims.json'
d2 = 'data/checked.json'

def check(data_path1, data_path3):
    checker = LLMChecker(
        model = 'bedrock/meta.llama3-1-70b-instruct-v1:0', 
        batch_size=50,
    )
    
    data = json.load(open(data_path1))

    for d in data:
        question = replace_name(d["question"])
        ref = d["reference"]
        text1 = d["text1"]
        text2 = d["text2"]
        claims_ref = d["claims_ref"]
        claims_t1 = d["claims_t1"]
        claims_t2 = d["claims_t2"]

        
        batch_claims = [claims_ref, claims_t1, claims_t2]
        references = [replace_name(ref), replace_name(text1), replace_name(text2)]

        batch_labels = checker.check(
            batch_claims=batch_claims,
            batch_questions=[question]*len(batch_claims),
            batch_references=[references] * len(batch_claims),
            is_joint=True,
            joint_check_num=5
        )
        d['labels'] = batch_labels

    with open(data_path3, 'w') as file:
        json.dump(data, file, indent=4)
        
check(d1,d2)


  0%|          | 0/1 [00:00<?, ?it/s]

litellm.RateLimitError: BedrockException - {"message":"Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again."} [sleep 10 seconds]


  0%|          | 0/1 [00:00<?, ?it/s]

litellm.RateLimitError: BedrockException - {"message":"Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again."} [sleep 10 seconds]


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

litellm.RateLimitError: BedrockException - {"message":"Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again."} [sleep 10 seconds]


  0%|          | 0/2 [00:00<?, ?it/s]

litellm.RateLimitError: BedrockException - {"message":"Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again."} [sleep 10 seconds]


100%|██████████| 1/1 [00:01<00:00,  1.86s/it]


In [1]:
import json

data_path1 = 'data/checked.json'
data_path2 = 'data/score.json'

score_map = {
    'Entailment': 1,
    'Neutral': 0,
    'Contradiction': 0
}

with open(data_path1) as f:
    data = json.load(f)

for d in data:
    labels = d['labels']
    ref_label = labels[0]
    t1_label = labels[1]
    t2_label = labels[2]

    sum1 = 0
    sum2 = 0
    ref_to_t1 = 0
    ref_to_t2 = 0
    t1_to_ref = 0
    t2_to_ref = 0   

    for label in ref_label:
        sum1 += 1
        sum2 += 1
        ref_to_t1 += score_map[label[1]]
        ref_to_t2 += score_map[label[2]]
    for label in t1_label:
        sum1 += 1
        t1_to_ref += score_map[label[0]]
    for label in t2_label:
        sum2 += 1
        t2_to_ref += score_map[label[0]]

    d['ref_t1_score'] = (ref_to_t1 + t1_to_ref) / sum1  
    d['ref_t2_score'] = (ref_to_t2 + t2_to_ref) / sum2

with open(data_path2, 'w') as f:
    json.dump(data, f, indent=4)
